In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import os
import pathlib
import cv2

import glob
from PIL import Image
import random

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from prettytable import PrettyTable

from sklearn.metrics import confusion_matrix
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score


import tensorflow as tf
from tensorflow import keras
from keras.models import Sequential, Model, load_model
from keras.layers import Dense, Dropout, Flatten, Activation, Conv2D, MaxPooling2D, Input, LSTM, TimeDistributed, BatchNormalization
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.regularizers import l1,l2
from tensorflow.keras.utils import to_categorical

from datetime import datetime

%matplotlib inline

In [ ]:
list_ = os.listdir('/content/drive/MyDrive/AI3 Project Group/DataFolder/full-heatstamps/') # dir is your directory path
number_files = len(list_)
print(number_files)

list_ = os.listdir('/content/drive/MyDrive/AI3 Project Group/DataFolder/small-heatstamps/') # dir is your directory path
number_files = len(list_)
print(number_files)

5087
311


In [ ]:
def load_images_from_folder(folder):
    print("Reading Images")
    df = pd.DataFrame()

    frame_index=0
    for filename in glob.glob(folder):
        #print(f'Filename : {filename}')

        idx_start=filename.find("heatstamps")
        assert (idx_start>-1), 'Directory names are not correct!'
        idx_end= filename.find("_")
        exp_id = filename[idx_start+11:idx_end]

        idx_start = idx_end+1
        idx_end = filename.find("_", idx_start)
        time = filename[idx_start:idx_end]

        idx_start = idx_end+1
        hrr = float(filename[idx_start:-4])

        df_row = pd.DataFrame(data=np.array([[exp_id, time, hrr, filename, frame_index]]), 
                              columns=['expID','time','hrr', 'file_path', 'frame_index'])
        df = pd.concat([df,df_row], ignore_index=True)
        frame_index += 1
                                       
    print("Finished Reading Images")
    # print("Total number of images read =", x_image.shape[0], end='\n      ')
    
    df = df.astype({"expID": int, "time": int, "hrr":float, 'file_path':str, 'frame_index': int}, errors='raise') 
    df["datetime"] = df.expID.apply(lambda x: datetime.fromtimestamp(int(x)))
    
    print("Creating df_exp pandaframe")
    df_exp = pd.DataFrame()
    expID_list = list(df.expID.unique())
    for exp in expID_list:
        df_temp = df[df.expID==exp].sort_values(by="time")
        time_list = df_temp.time.tolist()
        hrr_list = df_temp.hrr.tolist()
        filename_list = df_temp.file_path.tolist()
        frameindex_list = df_temp.frame_index.tolist()
        df_row = pd.DataFrame(data=np.array([[exp, time_list, hrr_list, filename_list, frameindex_list, len(df_temp)]]),
                              columns=['expID', 'time_list', 'hrr_list', 'filename_list', 'frameindex_list', 'num_of_frames'])
        df_exp = pd.concat([df_exp,df_row], ignore_index=True)
    df_exp = df_exp.astype({"expID": int,  'num_of_frames': int}, errors='raise') 
    return df, df_exp

In [ ]:
folder_name = '/content/drive/MyDrive/AI3 Project Group/DataFolder/full-heatstamps/*.jpg' # /content/drive/MyDrive/ai3/DataFolder/full-heatstamps small-heatstamps
size_x = 1080
size_y = 1920
df, df_exp =load_images_from_folder(folder_name)

Reading Images
Finished Reading Images
Creating df_exp pandaframe


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:41: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.


In [ ]:
df_exp.head()

,expID,time_list,hrr_list,filename_list,frameindex_list,num_of_frames
0,1492091485,"[0, 100, 200, 300, 400, 500, 600, 700, 800, 90...","[329.91, 2373.3, 8395.7, 7242.1, 8244.0, 6152....",[/content/drive/MyDrive/AI3 Project Group/Data...,"[1888, 1881, 1943, 1960, 1977, 1971, 1993, 4, ...",144
1,1492523586,"[0, 100, 200, 300, 400, 500, 600, 700, 800, 90...","[-0.63, -3.51, 2.66, -9.88, -3.85, -12.41, 31....",[/content/drive/MyDrive/AI3 Project Group/Data...,"[31, 50, 82, 102, 108, 111, 126, 152, 157, 169...",140
2,1490104501,"[0, 100, 200, 300, 400, 500, 600, 700, 800, 90...","[13.55, -8.2, 19.29, 91.71, 133.87, 244.64, 30...",[/content/drive/MyDrive/AI3 Project Group/Data...,"[1765, 1778, 1799, 1819, 1814, 1836, 1847, 185...",118
3,1510150535,"[0, 100, 200, 300, 400, 500]","[1.72, -0.83, -4.03, -1.29, -0.86, -1.37]",[/content/drive/MyDrive/AI3 Project Group/Data...,"[168, 160, 189, 184, 180, 175]",6
4,1510153921,"[0, 100, 200, 300, 400, 500, 600, 700, 800, 90...","[3.18, 5339.3, 583.87, 1132.3, 977.15, 1151.4,...",[/content/drive/MyDrive/AI3 Project Group/Data...,"[177, 176, 183, 179, 181, 186, 182, 190, 187, ...",13


In [ ]:
train_df, val_df = train_test_split(df_exp[df_exp.num_of_frames<30], train_size = 0.8)

In [ ]:
AUTOTUNE = tf.data.AUTOTUNE

train_x = tf.ragged.constant( train_df.filename_list.to_numpy() )
train_y = tf.ragged.constant( train_df.hrr_list.to_numpy() )

val_x = tf.ragged.constant( val_df.filename_list.to_numpy() )
val_y = tf.ragged.constant( val_df.hrr_list.to_numpy() )

train_data  = tf.data.Dataset.from_tensor_slices( (train_x, train_y) )
validation_data = tf.data.Dataset.from_tensor_slices( (val_x, val_y) )

BATCH_SIZE = 1

size_x = 1080
size_y = 1920

@tf.function
def get_img(file_path):
  # print(file_path[0].numpy())
  # read data
  img = tf.io.read_file(file_path)
  img = tf.image.decode_jpeg(img)
  img = tf.image.resize(img, [540, 960])
  img = img/255.

  return img

@tf.function
def process_data(filepath_list, target_list):
  imgs_tensor =  filepath_list.with_flat_values(tf.map_fn(get_img, filepath_list.flat_values, fn_output_signature=tf.float32)) # rt.with_flat_values(map_fn(fn, rt.flat_values))

  return imgs_tensor.to_tensor(), tf.expand_dims(target_list.to_tensor(),2)    # target_list.to_tensor()



train_data = train_data.shuffle(buffer_size = len(train_df))
train_data = train_data.batch(batch_size = BATCH_SIZE)
train_data = train_data.map(process_data, num_parallel_calls = AUTOTUNE)
# train_data = train_data.map(lambda x,y: (tf.cast(x, tf.float32),y))
# train_data = train_data.map(pad_tensors)
# train_data = train_data.map(lambda x,y: (x.to_tensor(default_value = 0, shape = [None, None, None, None]), y.to_tensor(default_value = 0, shape = [None])))
train_data = train_data.prefetch(buffer_size = AUTOTUNE)


validation_data = validation_data.shuffle(buffer_size = len(val_df))
validation_data = validation_data.batch(batch_size = BATCH_SIZE)
validation_data = validation_data.map(process_data, num_parallel_calls = AUTOTUNE)
# validation_data = validation_data.map(lambda x,y: (tf.cast(x, tf.float32),y))
# validation_data = validation_data.map(pad_tensors)
# train_data = train_data.map(lambda x,y: (x.to_tensor(default_value = 0, shape = [None, None, None, None]), y.to_tensor(default_value = 0, shape = [None])))
validation_data = validation_data.prefetch(buffer_size = AUTOTUNE)

In [ ]:
for x,y in train_data.take(1):
  print(x.shape)
  print(y.shape)
  print(y)

(1, 11, 540, 960, 3)
(1, 11, 1)
tf.Tensor(
[[[1.2500e+00]
  [3.5577e+02]
  [1.8551e+03]
  [7.0198e+02]
  [1.7819e+02]
  [7.5200e+01]
  [3.2010e+01]
  [2.1660e+01]
  [1.1230e+01]
  [8.1800e+00]
  [2.4800e+00]]], shape=(1, 11, 1), dtype=float32)


In [ ]:
max_len = 161
initialiser = 'glorot_uniform'
reg_lambda  = 0.001

input = Input(shape=(None,540,960,3))

x = TimeDistributed(Conv2D(8, (5, 5), strides=(2, 2), padding='same'))(input)
x = TimeDistributed(Activation('relu'))(x)

x = TimeDistributed(Conv2D(8, (3, 3), strides=(2, 2), padding='same'))(x)
x = TimeDistributed(Activation('relu'))(x)

x = TimeDistributed(MaxPooling2D(),name='First_Maxpool')(x)
#------------------------------------------------------------------------------------------------------------------------------------------------
x = TimeDistributed(Conv2D(16, (3, 3), strides=(2, 2), padding='same'))(x)
x = TimeDistributed(BatchNormalization())(x)
x = TimeDistributed(Activation('relu'))(x)

x = TimeDistributed(Conv2D(16, (3, 3), strides=(2, 2), padding='same'))(x)
x = TimeDistributed(Activation('relu'))(x)

x = TimeDistributed(MaxPooling2D(),name='Second_Maxpool')(x)
#------------------------------------------------------------------------------------------------------------------------------------------------
# x = TimeDistributed(Conv2D(32, (3, 3), strides=(2, 2), padding='same'))(x)
# x = TimeDistributed(BatchNormalization())(x)
# x = TimeDistributed(Activation('relu'))(x)

# x = TimeDistributed(MaxPooling2D(),name='Third_Maxpool')(x)
#------------------------------------------------------------------------------------------------------------------------------------------------
x = TimeDistributed(Flatten(),name='Flatten')(x)
x = LSTM(16,return_sequences=True,dropout=0.3)(x)
output = TimeDistributed(Dense(1, activation="linear"))(x)

model = Model(inputs=input,outputs=output)

model.summary()

Model: "model_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_2 (InputLayer)        [(None, None, 540, 960,   0         
                             3)]                                 
                                                                 
 time_distributed_10 (TimeDi  (None, None, 270, 480, 8  608      
 stributed)                  )                                   
                                                                 
 time_distributed_11 (TimeDi  (None, None, 270, 480, 8  0        
 stributed)                  )                                   
                                                                 
 time_distributed_12 (TimeDi  (None, None, 135, 240, 8  584      
 stributed)                  )                                   
                                                                 
 time_distributed_13 (TimeDi  (None, None, 135, 240, 8  0  

In [ ]:
model.compile(loss='mse',optimizer='adam',metrics=['mae'])

In [ ]:
model_history = model.fit(train_data,epochs=10,validation_data=validation_data)

Epoch 1/10
101/101 [==============================] - 22s 181ms/step - loss: 1375062.1250 - mae: 357.4597 - val_loss: 360100.5625 - val_mae: 262.5170
Epoch 2/10
101/101 [==============================] - 19s 180ms/step - loss: 1374782.2500 - mae: 357.3521 - val_loss: 359828.3438 - val_mae: 262.4347
Epoch 3/10
101/101 [==============================] - 16s 159ms/step - loss: 1374535.8750 - mae: 357.3166 - val_loss: 359579.1562 - val_mae: 262.3636
Epoch 4/10
101/101 [==============================] - 16s 160ms/step - loss: 1374296.7500 - mae: 357.2861 - val_loss: 359376.7812 - val_mae: 262.3134
Epoch 5/10
101/101 [==============================] - 18s 171ms/step - loss: 1374061.6250 - mae: 357.2633 - val_loss: 359175.1562 - val_mae: 262.2724
Epoch 6/10
101/101 [==============================] - 14s 136ms/step - loss: 1373821.1250 - mae: 357.2485 - val_loss: 358952.4688 - val_mae: 262.2274
Epoch 7/10
101/101 [==============================] - 15s 144ms/step - loss: 1373579.2500 - mae: 357

In [ ]:
model.save("/content/drive/MyDrive/cnn_lstm_4")

INFO:tensorflow:Assets written to: /content/drive/MyDrive/cnn_lstm_4/assets


INFO:tensorflow:Assets written to: /content/drive/MyDrive/cnn_lstm_4/assets


In [ ]:
mae = 0.0
count = 0.0

pred_flatten = []
true_flatten = []
for x,y in validation_data.take(len(validation_data)):
  y_pred = model.predict(x)
  #print(y_pred.shape, y.shape)
  #break
  for i,j in zip(y_pred,y):
    i=i.reshape(-1)
    j=tf.reshape(j,-1)
    args = (j.numpy() != 0.0)
    pred_flatten.extend(i.reshape(-1)[args])
    true_flatten.extend(j.numpy()[args])

for x, y in zip(pred_flatten,true_flatten):
  mae = mae + np.abs(x-y)

print(f'Mean absolute error is: {mae/len(pred_flatten)}')

Mean absolute error is: 263.337819563685
